# DBSCAN - visualization

Beata Baczynska

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import seaborn as sns
sns.set()

from ipywidgets import interact
import ipywidgets as widgets

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [74]:
%cd "/content/drive/My Drive/Colab Notebooks/Omdena"

/content/drive/My Drive/Colab Notebooks/Omdena


In [0]:
df = pd.read_csv("OUTPUT_WBI_exposer_cyclones_v2.csv", sep=";")

In [76]:
df.columns

Index(['SID', 'NAME', 'ISO', 'YEAR', 'BASIN', 'SUB BASIN', 'NATURE',
       'ISO_TIME', 'COORDS', 'TOTAL_HRS', 'DAY_HRS', 'NIGHT_HRS', 'USA_SSHS',
       'WIND_CALC_MEAN', 'PRES_CALC_MEAN', 'STORM_SPD_MEAN', 'STORM_DR_MEAN',
       'V_LAND_KN', '34KN_ASSETS', '64KN_ASSETS', '96KN_ASSETS', 'CPI',
       'TOTAL_DAMAGE_(000$)', 'TOTAL_DEATHS',
       'Air transport, freight (million ton-km)',
       'Arable land (hectares per person)', 'Cereal yield (kg per hectare)',
       'Food production index (2004-2006 = 100)', 'GDP growth (annual %)',
       'GDP per capita (constant 2010 US$)', 'Net flows from UN agencies US$',
       'Life expectancy at birth, total (years)',
       'Mobile cellular subscriptions (per 100 people)',
       'Population density (people per sq. km of land area)',
       'Adjusted savings: education expenditure (% of GNI)',
       'Rural population (% of total population)', 'Income_level_Final',
       'pop_max_34_adj', 'pop_max_50_adj', 'pop_max_64_adj', 'TOTAL_AFFEC

In [101]:
# choosing some numeric features
cols = ['YEAR', 'TOTAL_HRS', 'NIGHT_HRS', 'USA_SSHS', 'WIND_CALC_MEAN', 'PRES_CALC_MEAN', 'STORM_SPD_MEAN', 'STORM_DR_MEAN', 
        'V_LAND_KN', 'CPI', 'TOTAL_DAMAGE_(000$)', 'TOTAL_DEATHS', 'Air transport, freight (million ton-km)', 
        'Arable land (hectares per person)', 'Cereal yield (kg per hectare)', 'Food production index (2004-2006 = 100)', 
        'GDP growth (annual %)', 'GDP per capita (constant 2010 US$)', 'Net flows from UN agencies US$', 
        'Life expectancy at birth, total (years)', 'Mobile cellular subscriptions (per 100 people)', 
        'Population density (people per sq. km of land area)', 'Adjusted savings: education expenditure (% of GNI)', 
        'Rural population (% of total population)', 'pop_max_34_adj', 'pop_max_50_adj', 'pop_max_64_adj', 'TOTAL_AFFECTED']
df_cols = df[cols]
df_cols

,YEAR,TOTAL_HRS,NIGHT_HRS,USA_SSHS,WIND_CALC_MEAN,PRES_CALC_MEAN,STORM_SPD_MEAN,STORM_DR_MEAN,V_LAND_KN,CPI,TOTAL_DAMAGE_(000$),TOTAL_DEATHS,"Air transport, freight (million ton-km)",Arable land (hectares per person),Cereal yield (kg per hectare),Food production index (2004-2006 = 100),GDP growth (annual %),GDP per capita (constant 2010 US$),Net flows from UN agencies US$,"Life expectancy at birth, total (years)",Mobile cellular subscriptions (per 100 people),Population density (people per sq. km of land area),Adjusted savings: education expenditure (% of GNI),Rural population (% of total population),pop_max_34_adj,pop_max_50_adj,pop_max_64_adj,TOTAL_AFFECTED
46,1970,21,21,4,106.551282,933.961538,11.076923,289.846154,152.4,15.186335,2353.0,137.0,21.700001,0.130434,1350.3,34.36,3.764605,1257.036642,4.660000e+06,63.155,0.000000,120.041554,2.757732,67.021,5.951658e+06,1.357762e+06,2.340927e+05,145200
47,1970,0,0,4,99.133333,937.875000,16.100000,273.800000,138.2,15.186335,97656.0,786.0,21.700001,0.130434,1350.3,34.36,3.764605,1257.036642,4.660000e+06,63.155,0.000000,120.041554,2.757732,67.021,1.883701e+07,1.086817e+07,3.486536e+05,432075
49,1971,9,9,1,72.000000,972.115385,14.230769,285.461538,118.1,15.838255,590.0,57.0,31.600000,0.126026,1272.2,35.81,5.428631,1287.654850,7.110000e+06,63.190,0.000000,123.548833,2.800000,66.507,1.850320e+07,3.913293e+06,8.733250e+05,32503
52,1972,18,18,-1,53.162500,986.468750,9.083333,188.958333,109.0,16.356525,23000.0,204.0,36.900002,0.123934,1163.9,35.26,5.446791,1319.347738,6.160000e+06,63.202,0.000000,127.148783,2.205623,65.988,1.078461e+07,3.620051e+06,1.385058e+06,28
54,1972,6,2,1,65.064103,981.730769,16.076923,299.000000,100.5,16.356525,15000.0,131.0,36.900002,0.123934,1163.9,35.26,5.446791,1319.347738,6.160000e+06,63.202,0.000000,127.148783,2.205623,65.988,2.306806e+07,1.684023e+07,9.618907e+06,385000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,2015,6,5,-1,32.059524,999.791667,13.642857,294.000000,39.2,92.708822,1300.0,2.0,604.508812,0.054743,3556.1,116.33,6.066549,2605.493599,1.968888e+07,70.644,115.399439,342.466419,1.840000,53.716,3.692360e+07,7.771056e+06,2.010055e+06,3500
883,2015,21,9,1,71.403846,966.884615,10.615385,313.923077,115.8,92.708822,4200000.0,20.0,19805.630000,0.086784,5896.2,135.91,6.905317,6484.435948,5.549183e+07,75.928,91.835388,146.057642,1.790000,44.500,6.083631e+07,3.714914e+07,1.544625e+07,78300
884,2015,12,12,-1,62.467742,972.934896,5.000000,166.718750,126.5,92.708822,210985.0,51.0,604.508812,0.054743,3556.1,116.33,6.066549,2605.493599,1.968888e+07,70.644,115.399439,342.466419,1.840000,53.716,4.921537e+07,1.735112e+07,8.819082e+06,2898590
885,2015,21,21,5,114.875000,935.375000,17.250000,23.875000,151.6,92.708822,823000.0,14.0,714.003211,0.188957,3470.1,119.88,3.287992,10037.201490,6.867377e+06,74.904,88.371756,62.685901,5.212839,20.715,9.389196e+06,7.984246e+05,2.575739e+05,15000


In [102]:
df_cols_dict = {}
i = 0
for x in cols:
  df_cols_dict[i] = x
  i+=1

df_cols_dict

{0: 'YEAR',
 1: 'TOTAL_HRS',
 2: 'NIGHT_HRS',
 3: 'USA_SSHS',
 4: 'WIND_CALC_MEAN',
 5: 'PRES_CALC_MEAN',
 6: 'STORM_SPD_MEAN',
 7: 'STORM_DR_MEAN',
 8: 'V_LAND_KN',
 9: 'CPI',
 10: 'TOTAL_DAMAGE_(000$)',
 11: 'TOTAL_DEATHS',
 12: 'Air transport, freight (million ton-km)',
 13: 'Arable land (hectares per person)',
 14: 'Cereal yield (kg per hectare)',
 15: 'Food production index (2004-2006 = 100)',
 16: 'GDP growth (annual %)',
 17: 'GDP per capita (constant 2010 US$)',
 18: 'Net flows from UN agencies US$',
 19: 'Life expectancy at birth, total (years)',
 20: 'Mobile cellular subscriptions (per 100 people)',
 21: 'Population density (people per sq. km of land area)',
 22: 'Adjusted savings: education expenditure (% of GNI)',
 23: 'Rural population (% of total population)',
 24: 'pop_max_34_adj',
 25: 'pop_max_50_adj',
 26: 'pop_max_64_adj',
 27: 'TOTAL_AFFECTED'}

In [103]:
#Dropping nan values
df_cols.dropna(inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Scatter before normalization

In [104]:
@interact( x = cols, y = cols)
def scatt(x, y):
  plt.scatter(df[str(x)], df[str(y)])
  plt.xlabel(x)
  plt.ylabel(y)
  plt.show()

interactive(children=(Dropdown(description='x', options=('YEAR', 'TOTAL_HRS', 'NIGHT_HRS', 'USA_SSHS', 'WIND_C…

In [0]:
colors = ['royalblue', 'maroon', 'forestgreen', 'mediumorchid', 'tan', 'deeppink', 'olive', 
          'goldenrod', 'lightcyan', 'navy', 'gold', 'aliceblue', 'fuchsia',
          'lime', 'yellow', 'sienna', 'black', 'crimson', 'orange']
vectorizer = np.vectorize(lambda x: colors[x % len(colors)])

In [106]:
x = df_cols.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_cols_norm = pd.DataFrame(x_scaled)
df_cols_norm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,1.000000,1.000000,0.9,0.734487,0.925300,0.183746,0.839368,0.858601,0.000000,0.000019,0.000983,0.000544,0.037998,0.122183,0.000000,0.532399,0.019821,0.025361,0.486043,0.000000,0.098435,0.157686,0.759098,0.015835,0.004257,0.002873,4.901695e-03
1,0.000000,0.000000,0.045455,0.9,0.670552,0.929177,0.289629,0.791917,0.755102,0.000000,0.000781,0.005673,0.000544,0.037998,0.122183,0.000000,0.532399,0.019821,0.025361,0.486043,0.000000,0.098435,0.157686,0.759098,0.050118,0.034074,0.004279,1.458622e-02
2,0.022222,0.428571,0.454545,0.6,0.436692,0.963100,0.250227,0.826402,0.608601,0.008409,0.000005,0.000405,0.000792,0.036479,0.111280,0.013051,0.586293,0.020378,0.038694,0.486915,0.000000,0.101362,0.160850,0.752418,0.049230,0.012269,0.010718,1.097191e-03
3,0.044444,0.857143,0.863636,0.4,0.274333,0.977320,0.141723,0.541030,0.542274,0.015095,0.000184,0.001467,0.000925,0.035758,0.096161,0.008101,0.586881,0.020955,0.033524,0.487214,0.000000,0.104366,0.116357,0.745672,0.028694,0.011350,0.016998,8.777261e-07
4,0.044444,0.285714,0.136364,0.6,0.376912,0.972626,0.289143,0.866437,0.480321,0.015095,0.000120,0.000940,0.000925,0.035758,0.096161,0.008101,0.586881,0.020955,0.033524,0.487214,0.000000,0.104366,0.116357,0.745672,0.061375,0.052798,0.118050,1.299703e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,1.000000,0.285714,0.272727,0.4,0.092448,0.990519,0.237834,0.851651,0.033528,1.000000,0.000010,0.000007,0.015157,0.011918,0.430127,0.737804,0.606953,0.044378,0.107150,0.672537,0.777369,0.284050,0.088987,0.586163,0.098240,0.024364,0.024669,1.180879e-04
434,1.000000,1.000000,0.454545,0.6,0.431554,0.957917,0.174017,0.910566,0.591837,1.000000,0.033600,0.000137,0.496607,0.022958,0.756820,0.914041,0.634119,0.115018,0.301996,0.804121,0.618633,0.120146,0.085244,0.466375,0.161862,0.116471,0.189567,2.643238e-03
435,1.000000,0.571429,0.590909,0.4,0.354534,0.963912,0.055649,0.475265,0.669825,1.000000,0.001688,0.000361,0.015157,0.011918,0.430127,0.737804,0.606953,0.044378,0.107150,0.672537,0.777369,0.284050,0.088987,0.586163,0.130943,0.054400,0.108234,9.785255e-02
436,1.000000,1.000000,1.000000,1.0,0.806228,0.926700,0.313870,0.052859,0.852770,1.000000,0.006584,0.000094,0.017903,0.058162,0.418121,0.769757,0.516963,0.179718,0.037373,0.778621,0.595301,0.050572,0.341469,0.157222,0.024981,0.002503,0.003161,5.063129e-04


## Choosing paramethrs: eps and min_samples. Receiving number of clusters

In [107]:
@interact (eps = (0.1, 2.0, 0.1), min_samples = (2, 25, 1))
def for_clustering(eps, min_samples):
  m = DBSCAN(eps=eps, min_samples=min_samples)
  m.fit(df_cols_norm)

  global clusters
  clusters = m.labels_
  return "Number of clusters: " + str(len(np.unique(clusters)))

interactive(children=(FloatSlider(value=1.0, description='eps', max=2.0, min=0.1), IntSlider(value=13, descrip…

In [0]:
features_array = [('YEAR', 0) , ('TOTAL_HRS', 1), ('NIGHT_HRS', 2), ('USA_SSHS',3), ('WIND_CALC_MEAN',4), ('PRES_CALC_MEAN', 5),
 ('STORM_SPD_MEAN', 6), ('STORM_DR_MEAN', 7), ('V_LAND_KN', 8), ('CPI', 9), ('TOTAL_DAMAGE_(000$)',10), ('TOTAL_DEATHS',11),
 ('Air transport, freight (million ton-km)',12), ('Arable land (hectares per person)',13), ('Cereal yield (kg per hectare)',14),
 ('Food production index (2004-2006 = 100)',15), ('GDP growth (annual %)',16), ('GDP per capita (constant 2010 US$)',17),
 ('Net flows from UN agencies US$',18), ('Life expectancy at birth, total (years)',19),
 ('Mobile cellular subscriptions (per 100 people)',20), ('Population density (people per sq. km of land area)',21),
 ('Adjusted savings: education expenditure (% of GNI)',22), ('Rural population (% of total population)',23),
 ('pop_max_34_adj',24), ('pop_max_50_adj',25), ('pop_max_64_adj',26), ('TOTAL_AFFECTED', 27)]

## Scatter with clusters, after normalization

In [0]:
@interact (x = features_array, y = features_array)
def scatt(x, y):
  plt.scatter(df_cols_norm[x], df_cols_norm[y], c=vectorizer(clusters))
  plt.xlabel(df_cols_dict[x])
  plt.ylabel(df_cols_dict[y])
  plt.show()

interactive(children=(Dropdown(description='x', options=(('YEAR', 0), ('TOTAL_HRS', 1), ('NIGHT_HRS', 2), ('US…

In [114]:
@interact(eps = (0.1, 2.0, 0.1), min_samples = (2, 25, 1), x = features_array, y = features_array)
def both(eps, min_samples, x, y):
  print(for_clustering(eps, min_samples))
  print()
  scatt(x, y)

interactive(children=(FloatSlider(value=1.0, description='eps', max=2.0, min=0.1), IntSlider(value=13, descrip…